BA5A

In [1]:
def find_min_coin(money, coins):
    min_coin_need = [float('inf')]*(money+1)
    min_coin_need[0] = 0
    
    for i in range(1, money+1):
        for coin in coins:
            if i >= coin:
                min_coin_need[i] = min(min_coin_need[i], min_coin_need[i-coin]+1)
    
    return min_coin_need[money] if min_coin_need[money] != float('inf') else -1

with open('rosalind_ba5a.txt') as fp:
    lines = fp.readlines()
    lines = [line.strip() for line in lines]
    money = int(lines[0])
    coins = [int(coin) for coin in lines[1].split(',')]
    
    min_coin_need = find_min_coin(money, coins)

print(min_coin_need)

975


BA5B

In [2]:
import numpy as np

def find_longest_path(n, m, down_matrix, right_matrix):
    longest_path = np.zeros((n+1, m+1))
    longest_path[0, 1:] = np.cumsum(right_matrix[0,:])
    longest_path[1:, 0] = np.cumsum(down_matrix[:,0])
    for i in range(1,n+1):
        for j in range(1,m+1):
            longest_path[i, j] = max(longest_path[i-1, j] + down_matrix[i-1,j], longest_path[i, j-1] + right_matrix[i, j-1])
    
    return longest_path[n, m]

with open('rosalind_ba5b.txt') as fp:
    lines = fp.readlines()
    lines = [line.strip() for line in lines]
    n, m = tuple(map(int, lines[0].split()))
    down_matrix = np.array([list(map(int, line.split(' '))) for line in lines[1:n+1]])
    right_matrix = np.array([list(map(int, line.split(' '))) for line in lines[n+2:]])
    longest_path = find_longest_path(n, m, down_matrix, right_matrix)
    
print(int(longest_path))
    
    

27


BA5C

In [ ]:
import pandas as pd
import numpy as np

def get_matrix(sequence1, sequence2):
    len1 = len(sequence1)
    len2 = len(sequence2)
    seq_matrix = pd.DataFrame(np.zeros((len1, len2)), index=list(sequence1), columns=list(sequence2))
    for base1 in seq_matrix.index:
        for base2 in seq_matrix.columns:
            if base1 == base2:
                seq_matrix.loc[base1, base2]=1
    
    return seq_matrix

def get_lls(sequence1, sequence2):
    len1 = len(sequence1)
    len2 = len(sequence2)
    seq_matrix = get_matrix(sequence1, sequence2)
    max_lcs = np.zeros((len1+1, len2+1))
    max_lcs[0,:] = 0
    max_lcs[:,0] = 0
    for i in range(1,len1+1):
        for j in range(1,len2+1):
            max_lcs[i, j] = max(max_lcs[i-1,j], max_lcs[i, j-1], max_lcs[i-1, j-1] + seq_matrix.iloc[i-1, j-1])
            
    i, j = len1, len2
    
    lcs = []
    seq1_new , seq2_new = [], []
    while i > 0 and j > 0:
        if seq_matrix.iloc[i - 1, j - 1] == 1 and max_lcs[i, j] == max_lcs[i - 1, j - 1] + 1:
            lcs.append(sequence1[i - 1])
            seq1_new.append(sequence1[i - 1])
            seq2_new.append(sequence2[j - 1])
            i -= 1
            j -= 1
        elif max_lcs[i, j] == max_lcs[i - 1, j]:
            seq1_new.append('-')
            seq2_new.append(sequence2[j - 1])
            
            i -= 1
        else:
            seq1_new.append(sequence1[i - 1])
            seq2_new.append('-')
            j -= 1
               
    return max_lcs, ''.join(lcs)[::-1], ''.join(seq1_new)[::-1], ''.join(seq2_new)[::-1]

with open('rosalind_ba5c.txt') as fp:
    lines = fp.readlines()
    lines = [line.strip() for line in lines]
    seq1 = lines[0]
    seq2 = lines[1]
    
max_lls, lcs , seq1_new, seq2_new = get_lls(seq1, seq2)
print(lcs)

BA5D

In [ ]:
from collections import defaultdict

path_dict_reverse = defaultdict(list)
weight_dict = defaultdict(lambda: -1)
memo = {}  # 用于缓存中间结果

def read_file(filename):
    global path_dict_reverse, weight_dict
    with open(filename) as fp:
        lines = fp.readlines()
        lines = [line.strip() for line in lines]
        start_node, end_node = lines[0], lines[1]
        for line in lines[2:]:
            path = tuple(line.split(':')[0].split('->'))
            weight = int(line.split(':')[1])
            weight_dict[path] = weight
    path_all = list(weight_dict.keys())
    for path in path_all:
        path_dict_reverse[path[1]].append(path[0])
    return start_node, end_node

def find_longest_path(node, start_node):
    global path_dict_reverse, weight_dict, memo
    if node == start_node:
        return 0
    if node in memo:
        return memo[node]
    if not path_dict_reverse[node]:
        return float('-inf')

    max_num = max([(find_longest_path(exnode, start_node) + weight_dict[(exnode, node)]) for exnode in path_dict_reverse[node]])
    memo[node] = max_num  # 缓存计算结果
    return max_num

def reverse_path(start_node, end_node):
    global path_dict_reverse, weight_dict
    path = [end_node]
    node = end_node
    while node != start_node:
        for ex_node in path_dict_reverse[node]:
            if weight_dict[(ex_node, node)] == find_longest_path(node, start_node) - find_longest_path(ex_node, start_node):
                path.append(ex_node)
                node = ex_node
                break    
    path.reverse()    
    return path

start_node, end_node = read_file('rosalind_ba5d.txt')

print(find_longest_path(node=end_node, start_node=start_node))
print('->'.join(reverse_path(start_node, end_node)))
       

55
11->40->43->45->46


BA5E

In [ ]:
import pandas as pd
import numpy as np

def read_blosum62(filename):
    df = pd.read_csv(filename, sep=',')
    acid_list = df.columns[1:]
    df = df.iloc[:, 1:]
    df.index = acid_list
    return df

def get_lls(seq1, seq2, blosum62, gap_penalty=-5):
    len1, len2 = len(seq1), len(seq2)
    max_lcs = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)
    result_index = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)

    max_lcs[0, :] = np.arange(0, (len2 + 1) * gap_penalty, gap_penalty)
    max_lcs[:, 0] = np.arange(0, (len1 + 1) * gap_penalty, gap_penalty)

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match_score = blosum62.loc[seq1[i - 1], seq2[j - 1]]
            scores = [
                max_lcs[i - 1, j] + gap_penalty, 
                max_lcs[i, j - 1] + gap_penalty,  
                max_lcs[i - 1, j - 1] + match_score  
            ]
            max_lcs[i, j] = max(scores)
            result_index[i, j] = np.argmax(scores)

    # 回溯以构建比对
    aligned_seq1, aligned_seq2 = traceback(seq1, seq2, result_index)
    return int(max_lcs[len1, len2]), aligned_seq1, aligned_seq2

def traceback(seq1, seq2, result_index):
    i, j = len(seq1), len(seq2)
    seq1_new, seq2_new = [], []
    while i > 0 or j > 0:
        if i > 0 and j > 0 and result_index[i, j] == 2:
            seq1_new.append(seq1[i - 1])
            seq2_new.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and result_index[i, j] == 0:
            seq1_new.append(seq1[i - 1])
            seq2_new.append('-')
            i -= 1
        elif j > 0:
            seq1_new.append('-')
            seq2_new.append(seq2[j - 1])
            j -= 1
    return ''.join(seq1_new[::-1]), ''.join(seq2_new[::-1])

# 读取输入数据并输出结果
with open('test.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

blosum62 = read_blosum62('blosum62_matrix.csv')
alignment_score, aligned_seq1, aligned_seq2 = get_lls(seq1, seq2, blosum62)

print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

8
PLEASANTLY
-MEA--N-LY


BA5F

In [ ]:
import pandas as pd
import numpy as np

def read_blosum62(filename):
    df = pd.read_csv(filename, sep=',')
    acid_list = df.columns[1:]
    df = df.iloc[:, 1:]
    df.index = acid_list
    return df

def get_local_alignment(seq1, seq2, blosum62, gap_penalty=-5):
    len1, len2 = len(seq1), len(seq2)
    max_lcs = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)
    result_index = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)

    max_lcs[0, :] = 0
    max_lcs[:, 0] = 0

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match_score = blosum62.loc[seq1[i - 1], seq2[j - 1]]
            scores = [
                0, 
                max_lcs[i - 1, j] + gap_penalty,
                max_lcs[i, j - 1] + gap_penalty, 
                max_lcs[i - 1, j - 1] + match_score  
            ]
            max_lcs[i, j] = max(scores)
            result_index[i, j] = np.argmax(scores)

    # 回溯以构建比对结果
    aligned_seq1, aligned_seq2, max_score = traceback(seq1, seq2, max_lcs, result_index)
    return max_lcs, aligned_seq1, aligned_seq2, max_score

def traceback(seq1, seq2, max_lcs, result_index):
    # 从最大得分的位置开始回溯
    i, j = np.unravel_index(np.argmax(max_lcs), max_lcs.shape)
    max_score = max_lcs[i, j]
    seq1_new, seq2_new = [], []
    
    while i > 0 and j > 0 and max_lcs[i, j] > 0: 
        if result_index[i, j] == 3:  
            seq1_new.append(seq1[i - 1])
            seq2_new.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif result_index[i, j] == 1: 
            seq1_new.append(seq1[i - 1])
            seq2_new.append('-')
            i -= 1
        elif result_index[i, j] == 2:  
            seq1_new.append('-')
            seq2_new.append(seq2[j - 1])
            j -= 1

    return ''.join(seq1_new[::-1]), ''.join(seq2_new[::-1]), max_score

# 读取输入数据并输出结果
with open('rosalind_ba5f.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

blosum62 = read_blosum62('PAM250.csv')
alignment_score, aligned_seq1, aligned_seq2, max_score = get_local_alignment(seq1, seq2, blosum62)

print(max_score)         # 最大得分
print(aligned_seq1)      # 对齐后的第一个序列
print(aligned_seq2)      # 对齐后的第二个序列

BA5G

In [ ]:
import pandas as pd
import numpy as np

def find_min_edits(seq1, seq2):
    len1 = len(seq1)
    len2 = len(seq2)
    edit_matrix = np.zeros((len1 + 1, len2 + 1))
    for i in range(len1 + 1):
        for j in range(len2 + 1):
            cost = 0 if seq1[i-1] == seq2[j-1] else 1
            edit_matrix[i, j] = min([edit_matrix[i-1, j]+1, edit_matrix[i, j-1]+1, edit_matrix[i-1, j-1]+cost])
            if i>0 and j==0:
                edit_matrix[i, j] = edit_matrix[i-1, j]+1
            if i==0 and j>0:
                edit_matrix[i, j] = edit_matrix[i, j-1]+1
    return int(edit_matrix[len1, len2])

with open('rosalind_ba5g.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()
print(find_min_edits(seq1, seq2))

1761


BA5H

In [ ]:
import pandas as pd
import numpy as np

def find_fitting_alignment(seq1, seq2, gap_penalty=-1):
    len1, len2 = len(seq1), len(seq2)
    max_lcs = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)
    result_index = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)

    # 初始化动态规划矩阵
    max_lcs[:, 0] = 0  # 第一列为 0，允许从任意位置开始匹配
    max_lcs[0, :] = np.arange(0, (len2 + 1) * gap_penalty, gap_penalty)

    # 填充动态规划矩阵
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match_score = 1 if seq1[i - 1] == seq2[j - 1] else -1
            scores = [
                max_lcs[i - 1, j] + gap_penalty,      # 删除
                max_lcs[i, j - 1] + gap_penalty,      # 插入
                max_lcs[i - 1, j - 1] + match_score   # 替换
            ]
            max_lcs[i, j] = max(scores)
            result_index[i, j] = np.argmax(scores)  # 记录方向

    max_i = np.argmax(max_lcs[:, len2])
    max_score = max_lcs[max_i, len2]

    aligned_seq1, aligned_seq2 = traceback(seq1, seq2, max_i, len2, result_index)
    return max_score, aligned_seq1, aligned_seq2

def traceback(seq1, seq2, i, j, result_index):
    seq1_new, seq2_new = [], []
    while j > 0 and i > 0:
        if i > 0 and j > 0 and result_index[i, j] == 2:
            seq1_new.append(seq1[i - 1])
            seq2_new.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and result_index[i, j] == 0:
            seq1_new.append(seq1[i - 1])
            seq2_new.append('-')
            i -= 1
        elif j > 0:
            seq1_new.append('-')
            seq2_new.append(seq2[j - 1])
            j -= 1
    return ''.join(seq1_new[::-1]), ''.join(seq2_new[::-1])

# 读取输入数据并输出结果
with open('test.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

alignment_score, aligned_seq1, aligned_seq2 = find_fitting_alignment(seq1, seq2)

print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

2
TAGGCTTA
TAGA-T-A


BA5I

In [ ]:
import pandas as pd
import numpy as np

def find_fitting_alignment(seq1, seq2, gap_penalty=-2):
    len1, len2 = len(seq1), len(seq2)
    max_lcs = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)
    result_index = np.zeros((len1 + 1, len2 + 1), dtype=np.int32)

    # 初始化动态规划矩阵
    max_lcs[:, 0] = 0  
    max_lcs[0, :] = 0

    # 填充动态规划矩阵
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match_score = 1 if seq1[i - 1] == seq2[j - 1] else -2
            scores = [
                max_lcs[i - 1, j] + gap_penalty,    
                max_lcs[i, j - 1] + gap_penalty,     
                max_lcs[i - 1, j - 1] + match_score  
            ]
            max_lcs[i, j] = max(scores)
            result_index[i, j] = np.argmax(scores)  
    # 矩阵的最后一行，说明seq1已经完全遍历了，对应不同的seq2填补gap长度
    max_j = np.argmax(max_lcs[len1, :])
    max_score = max_lcs[len1, max_j]

    aligned_seq1, aligned_seq2 = traceback(seq1, seq2, len1, max_j, result_index)
    return max_score, aligned_seq1, aligned_seq2

def traceback(seq1, seq2, i, j, result_index):
    seq1_new, seq2_new = [], []
    while j > 0 and i > 0:
        if i > 0 and j > 0 and result_index[i, j] == 2:
            seq1_new.append(seq1[i - 1])
            seq2_new.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and result_index[i, j] == 0:
            seq1_new.append(seq1[i - 1])
            seq2_new.append('-')
            i -= 1
        elif j > 0:
            seq1_new.append('-')
            seq2_new.append(seq2[j - 1])
            j -= 1
    return ''.join(seq1_new[::-1]), ''.join(seq2_new[::-1])

# 读取输入数据并输出结果
with open('rosalind_ba5i.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

alignment_score, aligned_seq1, aligned_seq2 = find_fitting_alignment(seq1, seq2)

print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

45
CA-CACCC-GCTTAA-CAGACAAGGAT---CCCTCTTTCGCCCTGTAAGCGAACAGCGACTGTTTGAGAGCACATGACTTTGG-ATGATATCGCC-CATCAGTTTATGCAACC-GCACTGCAA-CGGTTCGTCCCAACGTA-CT-T-AAAGT-GCGACAAAATTC-GACACCCCCAATTCTTCCACGCTCCTCCTTG-TAGTATCGTG-C-ACACATGTCCG-TAACTCGGTTATTCTTGCT-GCGAAATTA-CTAA-TA-TATTGTGAG
CATC-CCCAGCTTATGCGGACAAGGATTTTCCCCCG--CGCCCCATAAGGGG-C-GCGACTGTTT-A-AGCACATGACTGTGGGATGCCAT-GTTTCATCAGTTTATGCCAGCTGCACTTCCAGCAGCTC-TGCCAACG-AGCTGTGAAAGTCGCGGGAAGGTCCAG-CATCACCGAT-CTTGCACGCTCCTACTTGGTAGTATCGCGACTACACAT--CCGGTAACTTGGTT-TTCTTGCTTGAGAAATCAGCCACCTAATATTGTGA-


BA5J

In [ ]:
import pandas as pd
import numpy as np

def read_blosum62(filename):
    df = pd.read_csv(filename, sep=',')
    acid_list = df.columns[1:]
    df = df.iloc[:, 1:]
    df.index = acid_list
    return df

def get_lls(seq1, seq2, blosum62, open_gap_penalty=-11, continue_gap_penalty=-1):
    len1, len2 = len(seq1), len(seq2)

    s = np.full((len1 + 1, len2 + 1), -np.inf)
    e = np.full((len1 + 1, len2 + 1), -np.inf)
    f = np.full((len1 + 1, len2 + 1), -np.inf)
    s[0, 0] = 0

    for i in range(1, len1 + 1):
        f[i, 0] = open_gap_penalty + (i - 1) * continue_gap_penalty
        s[i, 0] = f[i, 0]

    for j in range(1, len2 + 1):
        e[0, j] = open_gap_penalty + (j - 1) * continue_gap_penalty
        s[0, j] = e[0, j]

    trace_s = np.zeros((len1 + 1, len2 + 1), dtype=int) 
    trace_e = np.zeros((len1 + 1, len2 + 1), dtype=int) 
    trace_f = np.zeros((len1 + 1, len2 + 1), dtype=int) 

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match_score = blosum62.loc[seq1[i-1], seq2[j-1]]

            score_from_s = s[i, j-1] + open_gap_penalty
            score_from_e = e[i, j-1] + continue_gap_penalty
            e[i, j] = max(score_from_s, score_from_e)
            if e[i, j] == score_from_s:
                trace_e[i, j] = 0 
            else:
                trace_e[i, j] = 1  

            score_from_s = s[i-1, j] + open_gap_penalty
            score_from_f = f[i-1, j] + continue_gap_penalty
            f[i, j] = max(score_from_s, score_from_f)
            if f[i, j] == score_from_s:
                trace_f[i, j] = 0 
            else:
                trace_f[i, j] = 1 

            score_from_s = s[i-1, j-1] + match_score
            score_from_e = e[i, j]
            score_from_f = f[i, j]
            s[i, j] = max(score_from_s, score_from_e, score_from_f)
            if s[i, j] == score_from_s:
                trace_s[i, j] = 0  
            elif s[i, j] == score_from_e:
                trace_s[i, j] = 1  
            else:
                trace_s[i, j] = 2  

    i, j = len1, len2
    if s[i, j] == e[i, j]:
        state = 'E'
    elif s[i, j] == f[i, j]:
        state = 'F'
    else:
        state = 'S'

    seq1_new, seq2_new = [], []
    while i > 0 or j > 0:
        if state == 'S':
            if trace_s[i, j] == 0:
                seq1_new.append(seq1[i - 1])
                seq2_new.append(seq2[j - 1])
                i -= 1
                j -= 1
                state = 'S'
            elif trace_s[i, j] == 1:
                state = 'E'
            else:
                state = 'F'
        elif state == 'E':
            seq1_new.append('-')
            seq2_new.append(seq2[j - 1])
            j -= 1
            if trace_e[i, j + 1] == 0:
                state = 'S'
            else:
                state = 'E'
        elif state == 'F':
            seq1_new.append(seq1[i - 1])
            seq2_new.append('-')
            i -= 1
            if trace_f[i + 1, j] == 0:
                state = 'S'
            else:
                state = 'F'

    seq1_new.reverse()
    seq2_new.reverse()

    return int(s[len1, len2]), ''.join(seq1_new), ''.join(seq2_new)

with open('rosalind_ba5j.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

blosum62 = read_blosum62('blosum62_matrix.csv')
alignment_score, aligned_seq1, aligned_seq2 = get_lls(seq1, seq2, blosum62)

print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

231
QHRVGKWCKRPKDYMVQIPISSAWQEWHRPIDDKA-----VERYFNHAAVEIYCYNWLQCWMMMWWDYESESIEQQYSAFLPHLHVGPFTDGSHK
QHRAGKWCKRWKDYMK--------AGWHRPIDDKQREAGHVER-DNHAAVEIYCYNW------MHWDY--ESIEQQYSAKQCLIH-KPFTDGSAK


BA5K

In [5]:
import numpy as np
import pandas as pd

def read_blosum62(filename):
    df = pd.read_csv(filename, sep=',')
    acid_list = df.columns[1:]
    df = df.iloc[:, 1:]
    df.index = acid_list
    return df

def forward_calculate(seq1, seq2, gap_penalty, blosum62):
    len1 = len(seq1)
    len2 = len(seq2)
    previous = np.arange(0, (len1 + 1) * gap_penalty, gap_penalty)
    for j in range(1, len2 + 1):
        current = np.zeros(len1 + 1)
        current[0] = gap_penalty * j
        for i in range(1, len1 + 1):
            match = previous[i - 1] + blosum62.loc[seq1[i - 1], seq2[j - 1]]
            delete = previous[i] + gap_penalty
            insert = current[i - 1] + gap_penalty
            current[i] = max(match, delete, insert)
        previous = current
    return previous

def backward_calculate(seq1, seq2, gap_penalty, blosum62):
    len1 = len(seq1)
    len2 = len(seq2)
    previous = np.arange(0, (len1 + 1) * gap_penalty, gap_penalty)
    for j in range(1, len2 + 1):
        current = np.zeros(len1 + 1)
        current[0] = gap_penalty * j
        for i in range(1, len1 + 1):
            match = previous[i - 1] + blosum62.loc[seq1[i-1], seq2[j-1]]
            delete = previous[i] + gap_penalty
            insert = current[i - 1] + gap_penalty
            current[i] = max(match, delete, insert)
        if j != len2:
            previous = current
    
    return previous, current

def find_middle_edge(seq1, seq2, blosum62, gap_penalty=-5):
    len1 = len(seq1)
    len2 = len(seq2)
    middle_col = len2 // 2

    forward_score = forward_calculate(seq1, seq2[:middle_col], gap_penalty, blosum62)

    reverse_seq1 = seq1[::-1]
    reverse_seq2 = seq2[middle_col:][::-1]
    previous_score, backward_score = backward_calculate(reverse_seq1, reverse_seq2, gap_penalty, blosum62)
    backward_score_reverse = backward_score[::-1]
    total_scores = forward_score + backward_score_reverse

    max_row = np.argmax(total_scores)
    max_row_real = len1 - max_row
    
    flag = ''
    if backward_score[max_row_real] == previous_score[max_row_real-1] + blosum62.loc[reverse_seq1[max_row_real-1], reverse_seq2[-1]]:
        flag = 'diag'
    elif backward_score[max_row_real] == backward_score[max_row_real - 1] + gap_penalty:
        flag = 'down'
    else:
        flag = 'right'
    
    # if flag == 'down':
    #     print(f'({max_row}, {middle_col}) ({max_row+1}, {middle_col})')
    # elif flag == 'right':
    #     print(f'({max_row}, {middle_col}) ({max_row}, {middle_col+1})')
    # else:
    #     print(f'({max_row}, {middle_col}) ({max_row+1}, {middle_col+1})')
        
    return (max_row, middle_col), flag

with open('rosalind_ba5k.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

blosum62 = read_blosum62('blosum62_matrix.csv')
find_middle_edge(seq1, seq2, blosum62, gap_penalty=-5)

((512, 510), 'diag')

BA5L

In [7]:
BLOSUM62 = {
    'A': {'A':  4, 'C':  0, 'D': -2, 'E': -1, 'F': -2, 'G':  0, 'H': -2, 'I': -1,
          'K': -1, 'L': -1, 'M': -1, 'N': -2, 'P': -1, 'Q': -1, 'R': -1, 'S':  1,
          'T':  0, 'V':  0, 'W': -3, 'Y': -2},
    'C': {'A':  0, 'C':  9, 'D': -3, 'E': -4, 'F': -2, 'G': -3, 'H': -3, 'I': -1,
          'K': -3, 'L': -1, 'M': -1, 'N': -3, 'P': -3, 'Q': -3, 'R': -3, 'S': -1,
          'T': -1, 'V': -1, 'W': -2, 'Y': -2},
    'D': {'A': -2, 'C': -3, 'D':  6, 'E':  2, 'F': -3, 'G': -1, 'H': -1, 'I': -3,
          'K': -1, 'L': -4, 'M': -3, 'N':  1, 'P': -1, 'Q':  0, 'R': -2, 'S':  0,
          'T': -1, 'V': -3, 'W': -4, 'Y': -3},
    'E': {'A': -1, 'C': -4, 'D':  2, 'E':  5, 'F': -3, 'G': -2, 'H':  0, 'I': -3,
          'K':  1, 'L': -3, 'M': -2, 'N':  0, 'P': -1, 'Q':  2, 'R':  0, 'S':  0,
          'T': -1, 'V': -2, 'W': -3, 'Y': -2},
    'F': {'A': -2, 'C': -2, 'D': -3, 'E': -3, 'F':  6, 'G': -3, 'H': -1, 'I':  0,
          'K': -3, 'L':  0, 'M':  0, 'N': -3, 'P': -4, 'Q': -3, 'R': -3, 'S': -2,
          'T': -2, 'V': -1, 'W':  1, 'Y':  3},
    'G': {'A':  0, 'C': -3, 'D': -1, 'E': -2, 'F': -3, 'G':  6, 'H': -2, 'I': -4,
          'K': -2, 'L': -4, 'M': -3, 'N':  0, 'P': -2, 'Q': -2, 'R': -2, 'S':  0,
          'T': -2, 'V': -3, 'W': -2, 'Y': -3},
    'H': {'A': -2, 'C': -3, 'D': -1, 'E':  0, 'F': -1, 'G': -2, 'H':  8, 'I': -3,
          'K': -1, 'L': -3, 'M': -2, 'N':  1, 'P': -2, 'Q':  0, 'R':  0, 'S': -1,
          'T': -2, 'V': -3, 'W': -2, 'Y':  2},
    'I': {'A': -1, 'C': -1, 'D': -3, 'E': -3, 'F':  0, 'G': -4, 'H': -3, 'I':  4,
          'K': -3, 'L':  2, 'M':  1, 'N': -3, 'P': -3, 'Q': -3, 'R': -3, 'S': -2,
          'T': -1, 'V':  3, 'W': -3, 'Y': -1},
    'K': {'A': -1, 'C': -3, 'D': -1, 'E':  1, 'F': -3, 'G': -2, 'H': -1, 'I': -3,
          'K':  5, 'L': -2, 'M': -1, 'N':  0, 'P': -1, 'Q':  1, 'R':  2, 'S':  0,
          'T': -1, 'V': -2, 'W': -3, 'Y': -2},
    'L': {'A': -1, 'C': -1, 'D': -4, 'E': -3, 'F':  0, 'G': -4, 'H': -3, 'I':  2,
          'K': -2, 'L':  4, 'M':  2, 'N': -3, 'P': -3, 'Q': -2, 'R': -2, 'S': -2,
          'T': -1, 'V':  1, 'W': -2, 'Y': -1},
    'M': {'A': -1, 'C': -1, 'D': -3, 'E': -2, 'F':  0, 'G': -3, 'H': -2, 'I':  1,
          'K': -1, 'L':  2, 'M':  5, 'N': -2, 'P': -2, 'Q':  0, 'R': -1, 'S': -1,
          'T': -1, 'V':  1, 'W': -1, 'Y': -1},
    'N': {'A': -2, 'C': -3, 'D':  1, 'E':  0, 'F': -3, 'G':  0, 'H':  1, 'I': -3,
          'K':  0, 'L': -3, 'M': -2, 'N':  6, 'P': -2, 'Q':  0, 'R':  0, 'S':  1,
          'T':  0, 'V': -3, 'W': -4, 'Y': -2},
    'P': {'A': -1, 'C': -3, 'D': -1, 'E': -1, 'F': -4, 'G': -2, 'H': -2, 'I': -3,
          'K': -1, 'L': -3, 'M': -2, 'N': -2, 'P':  7, 'Q': -1, 'R': -2, 'S': -1,
          'T': -1, 'V': -2, 'W': -4, 'Y': -3},
    'Q': {'A': -1, 'C': -3, 'D':  0, 'E':  2, 'F': -3, 'G': -2, 'H':  0, 'I': -3,
          'K':  1, 'L': -2, 'M':  0, 'N':  0, 'P': -1, 'Q':  5, 'R':  1, 'S':  0,
          'T': -1, 'V': -2, 'W': -2, 'Y': -1},
    'R': {'A': -1, 'C': -3, 'D': -2, 'E':  0, 'F': -3, 'G': -2, 'H':  0, 'I': -3,
          'K':  2, 'L': -2, 'M': -1, 'N':  0, 'P': -2, 'Q':  1, 'R':  5, 'S': -1,
          'T': -1, 'V': -3, 'W': -3, 'Y': -2},
    'S': {'A':  1, 'C': -1, 'D':  0, 'E':  0, 'F': -2, 'G':  0, 'H': -1, 'I': -2,
          'K':  0, 'L': -2, 'M': -1, 'N':  1, 'P': -1, 'Q':  0, 'R': -1, 'S':  4,
          'T':  1, 'V': -2, 'W': -3, 'Y': -2},
    'T': {'A':  0, 'C': -1, 'D': -1, 'E': -1, 'F': -2, 'G': -2, 'H': -2, 'I': -1,
          'K': -1, 'L': -1, 'M': -1, 'N':  0, 'P': -1, 'Q': -1, 'R': -1, 'S':  1,
          'T':  5, 'V':  0, 'W': -2, 'Y': -2},
    'V': {'A':  0, 'C': -1, 'D': -3, 'E': -2, 'F': -1, 'G': -3, 'H': -3, 'I':  3,
          'K': -2, 'L':  1, 'M':  1, 'N': -3, 'P': -2, 'Q': -2, 'R': -3, 'S': -2,
          'T':  0, 'V':  4, 'W': -3, 'Y': -1},
    'W': {'A': -3, 'C': -2, 'D': -4, 'E': -3, 'F':  1, 'G': -2, 'H': -2, 'I': -3,
          'K': -3, 'L': -2, 'M': -1, 'N': -4, 'P': -4, 'Q': -2, 'R': -3, 'S': -3,
          'T': -2, 'V': -3, 'W': 11, 'Y':  2},
    'Y': {'A': -2, 'C': -2, 'D': -3, 'E': -2, 'F':  3, 'G': -3, 'H':  2, 'I': -1,
          'K': -2, 'L': -1, 'M': -1, 'N': -2, 'P': -3, 'Q': -1, 'R': -2, 'S': -2,
          'T': -2, 'V': -1, 'W':  2, 'Y':  7},
}

sigma = 5

def linear_space_alignment(v, w):
    alignment = ['', '']
    score = linear_space_alignment_helper(v, w, 0, len(v), 0, len(w), alignment)
    print(score)
    print(alignment[0])
    print(alignment[1])

def linear_space_alignment_helper(v, w, top, bottom, left, right, alignment):
    if left == right:
        for i in range(top, bottom):
            alignment[0] += v[i]
            alignment[1] += '-'
        return -sigma * (bottom - top)
    elif top == bottom:
        for j in range(left, right):
            alignment[0] += '-'
            alignment[1] += w[j]
        return -sigma * (right - left)
    elif bottom - top == 1 or right - left == 1:
        score, aligned_v, aligned_w = needleman_wunsch(v[top:bottom], w[left:right])
        alignment[0] += aligned_v
        alignment[1] += aligned_w
        return score
    else:
        mid_col = (left + right) // 2
        score_left = compute_forward(v, w, top, bottom, left, mid_col)
        score_right = compute_backward(v, w, top, bottom, mid_col, right)
        max_score = float('-inf')
        max_row = top
        for i in range(top, bottom + 1):
            total_score = score_left[i - top] + score_right[i - top]
            if total_score > max_score:
                max_score = total_score
                max_row = i
        score_top = linear_space_alignment_helper(v, w, top, max_row, left, mid_col, alignment)
        score_bottom = linear_space_alignment_helper(v, w, max_row, bottom, mid_col, right, alignment)
        return score_top + score_bottom

def compute_forward(v, w, top, bottom, left, mid_col):
    n = bottom - top  
    m = mid_col - left 
    current = [0] * (n + 1)
    for i in range(1, n + 1):
        current[i] = -sigma * i
    for j in range(1, m + 1):
        previous = current[:]
        current[0] = -sigma * j
        for i in range(1, n + 1):
            match = previous[i - 1] + BLOSUM62[v[top + i - 1]][w[left + j - 1]]
            delete = previous[i] - sigma
            insert = current[i - 1] - sigma
            current[i] = max(match, delete, insert)
    return current

def compute_backward(v, w, top, bottom, mid_col, right):
    n = bottom - top  
    m = right - mid_col  
    current = [0] * (n + 1)
    for i in range(1, n + 1):
        current[i] = -sigma * i
    for j in range(1, m + 1):
        previous = current[:]
        current[0] = -sigma * j
        for i in range(1, n + 1):
            match = previous[i - 1] + BLOSUM62[v[bottom - i]][w[right - j]]
            delete = previous[i] - sigma
            insert = current[i - 1] - sigma
            current[i] = max(match, delete, insert)
    return current[::-1]

def needleman_wunsch(v, w):
    n = len(v)
    m = len(w)
    s = [[0] * (m + 1) for _ in range(n + 1)]
    backtrack = [[''] * (m + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        s[i][0] = -sigma * i
        backtrack[i][0] = 'down'
    for j in range(1, m + 1):
        s[0][j] = -sigma * j
        backtrack[0][j] = 'right'
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = s[i - 1][j - 1] + BLOSUM62[v[i - 1]][w[j - 1]]
            delete = s[i - 1][j] - sigma
            insert = s[i][j - 1] - sigma
            s[i][j] = max(match, delete, insert)
            if s[i][j] == match:
                backtrack[i][j] = 'diag'
            elif s[i][j] == delete:
                backtrack[i][j] = 'down'
            else:
                backtrack[i][j] = 'right'
    aligned_v = ''
    aligned_w = ''
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrack[i][j] == 'diag':
            aligned_v = v[i - 1] + aligned_v
            aligned_w = w[j - 1] + aligned_w
            i -= 1
            j -= 1
        elif i > 0 and backtrack[i][j] == 'down':
            aligned_v = v[i - 1] + aligned_v
            aligned_w = '-' + aligned_w
            i -= 1
        else:  
            aligned_v = '-' + aligned_v
            aligned_w = w[j - 1] + aligned_w
            j -= 1
    return s[n][m], aligned_v, aligned_w

with open('rosalind_ba5l.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()

linear_space_alignment(seq1, seq2)

18092
LEPYQMCHPKAFSIFTEGNNWMQCCAKEGESVRPHLLHYCNRCKCHLDYSCLTSDKNPCWDQPIGCRTLDTRYQSLLMQRTMSH-MKTHDTDTTNLGQVSRPGAHVWGCNPGMNWMDVEMID-A---WC--RTNCTDVFETKCEFDWSN------T--W--KCH----PNSNYVRKFDGMRNNPQQ--N--NTEFRLILYGILLMVYDIRKH-DFICMYKGWTNRHQRGWRAYYSVRSRPMFWQLNT--GKDADGQMLDG--WCRNRYTLGRVIAWNYRGSDRYKGEGA-WS-GC-LILAINILVGDVSKKA-KHNLWVHPDSFIPSQYLVYHWQLLSSYVSFNWVNEHQLERCIGGSADRPIFYDDHAQF-------QCMWGSRVDQT--STCRCAC--HKDQDMWTISSNWFLMIWYMSPQTWVLFLAGQCRNFANPHQRHMNIWMSHVATLTIFICGGWKICKRC-EHGK------PICLYCLSAWYCI--GT---WQLVYIRFVEHEEWVFLKVNSGWVTFWFMHLYQCFMCTTRVQTWAQTHMQAPGHPYCYLWNHFYTECWEETYCIFVPCLMGI--V----HTK-YSDIVIVY-N---QHRMYPAAFYVFRMTITQLITCHEELTSCNGKFQSAN--RPTMQGSNCARNDEGIQPRLSKDPEVRDGHLS-Y--MVNYCKIVCKADRAFIPGFLSKAP---N-PNRKIPNMKWTSREGKISASTVLGCYMEVQGDKIIHYVLYFAVQMHIDHKPHPTICFDQQNCCYQSMVKQYYNSLCAISS-MMKCTFQRMQICLTQQGMHR--GNNTMAMLYHAPQPA---PRFKSNVDYNVVFCWWHEVQIHFPGTVSLHPPHDYDMNNVTYYIRQTQLAMAAPGEKRLSVLKFWGSSMLWAWAPRH-VEAAWQLGTWFDGVVHMKWKVHQKFTPMFVMRMLEYGFHREPNPNVHSPVVDEWLCWLIVQVQRKEGMIKCSFRFDASTERNFDNW

BA5M

In [13]:
def backtrack_alignment(seq1, seq2, seq3, backtrack):
    aligned_seq1 = []
    aligned_seq2 = []
    aligned_seq3 = []
    
    i, j, k = len(seq1), len(seq2), len(seq3)
    
    while i > 0 or j > 0 or k > 0:
        direction = backtrack[i, j, k]
        if direction == 0:
            aligned_seq1.append(seq1[i-1] if i > 0 else '-')
            aligned_seq2.append(seq2[j-1] if j > 0 else '-')
            aligned_seq3.append(seq3[k-1] if k > 0 else '-')
            i, j, k = i-1, j-1, k-1
        elif direction == 1:
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            aligned_seq3.append('-')
            i -= 1
        elif direction == 2:
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append(seq2[j-1])
            aligned_seq3.append('-')
            i, j = i-1, j-1
        elif direction == 3:
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            aligned_seq3.append(seq3[k-1])
            i, k = i-1, k-1
        elif direction == 4:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            aligned_seq3.append(seq3[k-1])
            j, k = j-1, k-1
        elif direction == 5:
            aligned_seq1.append('-')
            aligned_seq2.append('-')
            aligned_seq3.append(seq3[k-1])
            k -= 1
        elif direction == 6:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            aligned_seq3.append('-')
            j -= 1
    
    return ''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2)), ''.join(reversed(aligned_seq3))

def muti_seqs_dp(seq1, seq2, seq3):
    import numpy as np
    len1, len2, len3 = len(seq1), len(seq2), len(seq3)
    dp = np.zeros((len1+1, len2+1, len3+1), dtype=np.int32)
    backtrack = np.zeros((len1+1, len2+1, len3+1), dtype=np.int32)
    
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            for k in range(1, len3+1):
                match_score = 1 if seq1[i-1] == seq2[j-1] and seq2[j-1] == seq3[k-1] else 0
                all_possible_score = [
                    dp[i-1, j-1, k-1] + match_score,
                    dp[i-1, j, k],
                    dp[i-1, j-1, k],
                    dp[i-1, j, k-1],
                    dp[i, j-1, k-1],
                    dp[i, j, k-1],
                    dp[i, j-1, k]
                ]
                dp[i, j, k] = max(all_possible_score)
                backtrack[i, j, k] = np.argmax(all_possible_score)
    
    aligned_seq1, aligned_seq2, aligned_seq3 = backtrack_alignment(seq1, seq2, seq3, backtrack)
    return dp[len1, len2, len3], aligned_seq1, aligned_seq2, aligned_seq3

with open('rosalind_ba5m.txt') as fp:
    lines = fp.readlines()
    seq1 = lines[0].strip()
    seq2 = lines[1].strip()
    seq3 = lines[2].strip()


score, aligned_seq1, aligned_seq2, aligned_seq3 = muti_seqs_dp(seq1, seq2, seq3)
print(score)
print(aligned_seq1)
print(aligned_seq2)
print(aligned_seq3)
    
    
    
    

12
TCGGGGGAATC--AG----CA-AG-C---TG-CGCTA
TC-CGTTAA-C-CTGTAAT-AC-G-C--CTGA-G-GT
GC-AGTCAA-CTTTG-----A--GTCAAATG--G-CA


BA5N

In [20]:
from collections import defaultdict, deque

# 从文件解析图
def parse_graph_from_file(filename):
    graph = defaultdict(list)
    
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if '->' in line:
                src, dst = line.split('->')
                src = int(src.strip())
                destinations = [int(x.strip()) for x in dst.split(',')]
                graph[src].extend(destinations)
            else:
                continue

    all_nodes = set(graph.keys())
    for neighbors in graph.values():
        all_nodes.update(neighbors)
    for node in all_nodes:
        if node not in graph:
            graph[node] = []

    return graph

def topological_sort(graph):
    in_degree = defaultdict(int)
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    zero_in_degree = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while zero_in_degree:
        node = zero_in_degree.popleft()
        topo_order.append(node)
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                zero_in_degree.append(neighbor)

    return topo_order


graph = parse_graph_from_file('rosalind_ba5n.txt')
topo_order = topological_sort(graph)
print(', '.join(list(map(str, topo_order))))

0, 1, 17, 2, 20, 30, 4, 6, 7, 8, 39, 14, 3, 19, 10, 22, 5, 9, 21, 12, 13, 11, 32, 18, 33, 24, 27, 28, 29, 23, 34, 15, 26, 35, 37, 25, 16, 31, 36, 38
